# **How to use this notebook:**

Start with 1. - 1.2. On 1.3 Choose a specific network from the list printed by 1.1      
On 2. generate a new state (image) or paste the path to *projected_w.npz* (filesystem is on the left). Blank "seed" = random "seed"   
On 3. you can generate small perturbations of the original state. TYPE _plus_ adds small vectors to first 6 layers of the original state, _random_ replace them with some random numbers. On 3.1 you can save the state you like.   
On 4. you can generate images with the style of original, but with faces from random seeds. Use 4.1 to save.    
On 5. you can generate images with various styles and with original face (more or less). 5.1 used for saving

In [ ]:
#@title 1. Installs

from IPython.display import clear_output
%cd /content/
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch/
!pip install ninja

clear_output()
print('Done!')

In [ ]:
#@title # 1.1 Neural Networks handling
%cd /content/
from IPython.display import clear_output
import json

!curl https://raw.githubusercontent.com/dobrosketchkun/wd_network_zoo/main/wd_networks.json -O wd_networks.json


with open('/content/wd_networks.json', 'r') as f:
    jnetw = json.loads(f.read())

nn_list = list(jnetw.keys())

network_name = '' #@#param {type:"string"}
download_all = True #@#param {type:"boolean"}

if download_all:
    for nn in nn_list:
        url = jnetw[nn]
        id =  url.split('/')[-2]
        network_f_name = nn + '.pkl'
        !gdown --id $id -O $network_f_name
    clear_output()
    network_name = 'ffhq'
    print('Network:', network_name)
else:
    if not network_name:
        print('List of available networks:')
        print(', '.join(nn_list))
    elif network_name not in nn_list:
        print('Error, You can not use {}! Only names from this list are available:'.format(network_name))
        print(', '.join(nn_list))
    else:
        clear_output()
        print('Network:', network_name)
        url = jnetw[network_name]
        id =  url.split('/')[-2]
        network_f_name = network_name + '.pkl'
        !gdown --id $id -O $network_f_name

clear_output()

print('All networks are downloded.')
print('List of available networks:')
print(', '.join(nn_list))

In [ ]:
#@title # 1.2 Useful utility functions...
%cd /content/stylegan2-ada-pytorch/
import os
import re
from typing import List, Optional

import click
import dnnlib
import numpy as np
import PIL.Image
import torch
from io import BytesIO
import legacy



import argparse
import numpy as np
import PIL.Image
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio
import os
import pickle
from google.colab import files

def generate_images(zs, truncation_psi):
    # Gs_kwargs = dnnlib.EasyDict()
    # Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    # Gs_kwargs.randomize_noise = False
    # if not isinstance(truncation_psi, list):
    #     truncation_psi = [truncation_psi] * len(zs)
        
    imgs = []
    # for z_idx, z in log_progress(enumerate(zs), size = len(zs), name = "Generating images"):
    #     Gs_kwargs.truncation_psi = truncation_psi[z_idx]
    #     noise_rnd = np.random.RandomState(1) # fix noise
    #     tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
    #     images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
    #     imgs.append(PIL.Image.fromarray(images[0], 'RGB'))

    # for seed_idx, seed in enumerate(seeds):
    for z_idx, z in log_progress(enumerate(zs), size = len(zs), name = "Generating images"):
        # print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        # img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
        img = G(z, None, truncation_psi=truncation_psi, noise_mode=noise_mode)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        imgs.append(PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB'))
        # PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')
    return imgs

def generate_zs_from_seeds(seeds):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        # z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
        zs.append(z)
    return zs

# Generates a list of images, based on a list of seed for latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_from_seeds(seeds, truncation_psi):
    return generate_images(generate_zs_from_seeds(seeds), truncation_psi)

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location+ str(idx) + ".png"
    img.save(file)

def imshow(a, format='png', jpeg_fallback=True, save=False, id=None):
  a = np.asarray(a, dtype=np.uint8)
  str_file = BytesIO()
  PIL.Image.fromarray(a).save(str_file, format)
  #####
  if save:
      PIL.Image.fromarray(a).save(str(id[0]) + '.png', format)
  ####
  im_data = str_file.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print ('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format)
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

        
def clamp(x, minimum, maximum):
    return max(minimum, min(x, maximum))
    
def drawLatent(image,latents,x,y,x2,y2, color=(255,0,0,100)):
  buffer = PIL.Image.new('RGBA', image.size, (0,0,0,0))
   
  draw = ImageDraw.Draw(buffer)
  cy = (y+y2)/2
  draw.rectangle([x,y,x2,y2],fill=(255,255,255,180), outline=(0,0,0,180))
  for i in range(len(latents)):
    mx = x + (x2-x)*(float(i)/len(latents))
    h = (y2-y)*latents[i]*0.1
    h = clamp(h,cy-y2,y2-cy)
    draw.line((mx,cy,mx,cy+h),fill=color)
  return PIL.Image.alpha_composite(image,buffer)
             
  
def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols))) 
   return canvas

def convertZtoW(latent, truncation_psi=0.7, truncation_cutoff=9):
  dlatent = Gs.components.mapping.run(latent, None) # [seed, layer, component]
  dlatent_avg = Gs.get_var('dlatent_avg') # [component]
  for i in range(truncation_cutoff):
    dlatent[0][i] = (dlatent[0][i]-dlatent_avg)*truncation_psi + dlatent_avg
    
  return dlatent

def interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps) 
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )





In [ ]:
#@title # 1.3 Load style network
%cd /content/stylegan2-ada-pytorch/

import dnnlib
import legacy

network_name_ = '' #@param {type: "string"}

network_pkl = '/content/' + network_name_ + '.pkl'
noise_mode = 'const'


# print('Loading networks from "%s"...' % network_pkl)
device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore


In [ ]:
#@title # 2. Generate new or load a saved state (projected_w.npz)

from IPython.display import display
import copy
import uuid

seed = np.random.randint(2**32-1, size=1)


3771702234
seed = '' #@param {type:"string"}
save_npz = False #@param {type:"boolean"}
projected_w = '' #@param {type:"string"}

if not projected_w:
    try:
        seed = np.array([int(seed)])
    except Exception as e:
        # print('Something wrong with the seed\n', e)
        pass
    if not seed:
        seed = np.random.randint(2**32-1, size=1)
    print('Seed of this image is:', seed[0])

    truncation_psi = 0.7

    rand = np.random.RandomState(seed).randn(1, G.z_dim)

    all_z = np.stack(rand)
    all_w = G.mapping(torch.from_numpy(all_z).to(device), None)
    w_avg = G.mapping.w_avg
    all_w = w_avg + (all_w - w_avg) * truncation_psi

    all_images = G.synthesis(all_w, noise_mode=noise_mode)
    all_images = (all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
    im = PIL.Image.fromarray(all_images[0], 'RGB')

    display(createImageGrid([im], scale=0.5, rows=1))
else:
    ws = np.load(projected_w)['w']
    ws = torch.tensor(ws, device=device) # pylint: disable=not-callable
    all_w = ws
    orig_im_all_images = G.synthesis(all_w, noise_mode=noise_mode)
    orig_im_all_images = (orig_im_all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
    orig_im = PIL.Image.fromarray(orig_im_all_images[0], 'RGB') 
    display(createImageGrid([orig_im], scale=0.5, rows=1))   

if save_npz:
    np.savez('/content/orig_' + uuid.uuid4().hex + '_projected_w.npz', w=all_w.cpu().numpy())


In [ ]:
#@title # 3. Small variations

import random


# useWs = False
# projected_w = '/content/projected_w_nav.npz' 


TYPE = "plus"  #@param ['plus', 'random']



deli =  4 #@#param {type: "integer"}
start = -30000000/deli
end = 50000000/deli
plus = -0 * 10000000
plus2 = -0 * 10000000

deli_color = 5
start_color = -30000000/deli_color
end_color = 50000000/deli_color


# new = (np.random.randint(-4,5,(512))).astype('float32') 

pics_num = 15 #@param {type: "integer"}
rows = 4 #@param {type: "integer"}

pics_all_w_list_lv = []
imgs = []

# orig_im_ =  G.synthesis(all_w_orig, noise_mode=noise_mode)
# orig_im = (orig_im.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
# orig_im = PIL.Image.fromarray(orig_im[0], 'RGB')
if seed and not projected_w:
    orig_im_rand = np.random.RandomState(seed).randn(1, G.z_dim)
    orig_im_all_z = np.stack(orig_im_rand)
    orig_im_all_w = G.mapping(torch.from_numpy(orig_im_all_z).to(device), None)
    orig_im_w_avg = G.mapping.w_avg
    orig_im_all_w = orig_im_w_avg + (orig_im_all_w - orig_im_w_avg) * truncation_psi
    orig_im_all_images = G.synthesis(orig_im_all_w, noise_mode=noise_mode)
    orig_im_all_images = (orig_im_all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
    orig_im = PIL.Image.fromarray(orig_im_all_images[0], 'RGB')



for _ in range(pics_num):
  if projected_w:
    ws = np.load(projected_w)['w']
    ws = torch.tensor(ws, device=device) # pylint: disable=not-callable
    all_w = ws
    orig_im_all_images = G.synthesis(all_w, noise_mode=noise_mode)
    orig_im_all_images = (orig_im_all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
    orig_im = PIL.Image.fromarray(orig_im_all_images[0], 'RGB')

  else:
    all_w = copy.deepcopy(orig_im_all_w)
    

  if TYPE == 'plus':
    all_w[0][0] = all_w[0][0] + torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device) * random.choice([-1, 1]) * random.choice([0.1, 0.3, 0.5, 1])
    all_w[0][1] = all_w[0][1] + torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device) * random.choice([-1, 1]) * random.choice([0.1, 0.3, 0.5, 1])
    all_w[0][2] = all_w[0][2] + torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device) * random.choice([-1, 1]) * random.choice([0.1, 0.3, 0.5, 1])
    all_w[0][3] = all_w[0][3] + torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device) * random.choice([-1, 1]) * random.choice([0.1, 0.3, 0.5, 1])
    all_w[0][4] = all_w[0][4] + torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device) * random.choice([-1, 1]) * random.choice([0.1, 0.3, 0.5, 1])
    all_w[0][5] = all_w[0][5] + torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device) * random.choice([-1, 1]) * random.choice([0.1, 0.3, 0.5, 1])
    all_w[0][6] = all_w[0][6] + torch.from_numpy((np.random.randint(start_color,end_color,(512))/10000000).astype('float32')).to(device) * random.choice([-1, 1]) * random.choice([0.1, 0.3, 0.5, 1])
  
  elif TYPE == 'random':
    all_w[0][0] = torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device)
    all_w[0][1] = torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device)
    all_w[0][2] = torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device)
    all_w[0][3] = torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device)
    all_w[0][4] = torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device)
    all_w[0][5] = torch.from_numpy((np.random.randint(start + plus,end + plus2,(512))/10000000).astype('float32')).to(device)
    # all_w[0][6] = torch.from_numpy((np.random.randint(start_color,end_color,(512))/10000000).astype('float32')).to(device)
  else:
    raise Exception('Generation Type Error: TYPE should be only "plus" or "random".')

  ##(np.random.randint(-25283287,44716253,(512))/10000000).astype('float32') 
  pics_all_w_list_lv.append(all_w.cpu().numpy())

  all_images = G.synthesis(all_w, noise_mode=noise_mode)
  # print(all_images)
  all_images = (all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
  im = PIL.Image.fromarray(all_images[0], 'RGB')
  
  imgs.append(im)


# IPython.display.display(im)
createImageGrid([orig_im] + imgs, scale=0.4, rows=rows)

In [ ]:
#@title # 3.1 Choose one

import uuid

num =   0#@param {type:"integer"}
wwww = torch.from_numpy(pics_all_w_list_lv[num]).to(device)
all_images_ = G.synthesis(wwww, noise_mode=noise_mode)
# print(all_images)
all_images_ = (all_images_.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
im_ = PIL.Image.fromarray(all_images_[0], 'RGB')
display(createImageGrid([im_], scale=0.5, rows=1))

save_npz = False #@param {type:"boolean"}

if save_npz:
    np.savez('/content/' + uuid.uuid4().hex + '_projected_w_lv.npz', w=pics_all_w_list_lv[num])

In [ ]:
#@title # 4. Faces from random seeds

import copy


# useWs = False
# projected_w = '/content/projected_w_nav.npz'  



deli = 2
start = -30000000/deli
end = 50000000/deli
plus = -0 * 10000000
plus2 = -0 * 10000000

deli_color = 3
start_color = -30000000/deli_color
end_color = 50000000/deli_color


# new = (np.random.randint(-4,5,(512))).astype('float32') 

pics_num = 15 #@param {type: "integer"}
rows = 4 #@param {type: "integer"}

pics_all_w_list_ns = []
imgs = []

if seed:
    orig_im_rand = np.random.RandomState(seed).randn(1, G.z_dim)
    orig_im_all_z = np.stack(orig_im_rand)
    orig_im_all_w = G.mapping(torch.from_numpy(orig_im_all_z).to(device), None)
    orig_im_w_avg = G.mapping.w_avg
    orig_im_all_w = orig_im_w_avg + (orig_im_all_w - orig_im_w_avg) * truncation_psi
    orig_im_all_images = G.synthesis(orig_im_all_w, noise_mode=noise_mode)
    orig_im_all_images = (orig_im_all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
    orig_im = PIL.Image.fromarray(orig_im_all_images[0], 'RGB')



seeds = np.random.randint(2**32-1, size=pics_num)
truncation_psi = 0.7



if projected_w:
  ws = np.load(projected_w)['w']
  ws = torch.tensor(ws, device=device) # pylint: disable=not-callable
  all_w = ws
  orig_im_all_images = G.synthesis(all_w, noise_mode=noise_mode)
  orig_im_all_images = (orig_im_all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
  orig_im = PIL.Image.fromarray(orig_im_all_images[0], 'RGB')

else:
  all_w = copy.deepcopy(orig_im_all_w)



for seed in seeds:

  rand2 = np.random.RandomState(seed).randn(1, G.z_dim)
  all_z2 = np.stack(rand2)
  all_w2 = G.mapping(torch.from_numpy(all_z2).to(device), None)
  w_avg2 = G.mapping.w_avg
  all_w2 = w_avg2 + (all_w2 - w_avg2) * truncation_psi

  all_w[0][0] = all_w2[0][0]
  all_w[0][1] = all_w2[0][1]
  all_w[0][2] = all_w2[0][2]
  all_w[0][3] = all_w2[0][3]
  all_w[0][4] = all_w2[0][4]
  all_w[0][5] = all_w2[0][5]
  all_w[0][6] = all_w2[0][6]
  # all_w[0][7] = all_w2[0][7]
  # all_w[0][8] = all_w2[0][8]
  # all_w[0][9] = all_w2[0][9]
  # all_w[0][10] = all_w2[0][10]
  # all_w[0][11] = all_w2[0][11]
  # all_w[0][12] = all_w2[0][12]
  # all_w[0][13] = all_w2[0][13]
  # all_w[0][14] = all_w2[0][14]
  # all_w[0][15] = all_w2[0][15]
  # all_w[0][16] = all_w2[0][16]
  # all_w[0][17] = all_w2[0][17]
  ##(np.random.randint(-25283287,44716253,(512))/10000000).astype('float32') 

  pics_all_w_list_ns.append(all_w.cpu().numpy())

  all_images = G.synthesis(all_w, noise_mode=noise_mode)
  # print(all_images)
  all_images = (all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
  im = PIL.Image.fromarray(all_images[0], 'RGB')
  
  imgs.append(im)

# IPython.display.display(im)
createImageGrid([orig_im] + imgs, scale=0.4, rows=rows)

In [ ]:
#@title # 4.1 Choose one
import uuid

num =  0#@param {type:"integer"}
wwww = torch.from_numpy(pics_all_w_list_ns[num]).to(device)
all_images_ = G.synthesis(wwww, noise_mode=noise_mode)
# print(all_images)
all_images_ = (all_images_.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
im_ = PIL.Image.fromarray(all_images_[0], 'RGB')
display(createImageGrid([im_], scale=0.5, rows=1))

save_npz = False #@param {type:"boolean"}

if save_npz:
    np.savez('/content/' + uuid.uuid4().hex + '_projected_w_ns.npz', w=pics_all_w_list_ns[num])


In [ ]:
#@title # 5. Style from random seeds
import copy

# useWs = False
# projected_w = '/content/projected_w.npz' 


deli = 2
start = -30000000/deli
end = 50000000/deli
plus = -0 * 10000000
plus2 = -0 * 10000000

deli_color = 3
start_color = -30000000/deli_color
end_color = 50000000/deli_color


# new = (np.random.randint(-4,5,(512))).astype('float32') 

pics_num = 12 #@param {type: "integer"}
rows = 4 #@param {type: "integer"}
deeper_copy = True #@param {type:"boolean"}

pics_all_w_list_nst = []
imgs = []

if seed and not projected_w:
    orig_im_rand = np.random.RandomState(seed).randn(1, G.z_dim)
    orig_im_all_z = np.stack(orig_im_rand)
    orig_im_all_w = G.mapping(torch.from_numpy(orig_im_all_z).to(device), None)
    orig_im_w_avg = G.mapping.w_avg
    orig_im_all_w = orig_im_w_avg + (orig_im_all_w - orig_im_w_avg) * truncation_psi
    orig_im_all_images = G.synthesis(orig_im_all_w, noise_mode=noise_mode)
    orig_im_all_images = (orig_im_all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
    orig_im = PIL.Image.fromarray(orig_im_all_images[0], 'RGB')

# all_w = copy.deepcopy(orig_im_all_w)

if projected_w:
  ws = np.load(projected_w)['w']
  ws = torch.tensor(ws, device=device) # pylint: disable=not-callable
  all_w = ws
  orig_im_all_images = G.synthesis(all_w, noise_mode=noise_mode)
  orig_im_all_images = (orig_im_all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
  orig_im = PIL.Image.fromarray(orig_im_all_images[0], 'RGB')

else:
  all_w = copy.deepcopy(orig_im_all_w)



seeds = np.random.randint(2**32-1, size=pics_num)
truncation_psi = 0.7

for seed in seeds:

  rand3 = np.random.RandomState(seed).randn(1, G.z_dim)
  all_z3 = np.stack(rand3)
  all_w3 = G.mapping(torch.from_numpy(all_z3).to(device), None)
  w_avg3 = G.mapping.w_avg
  all_w3 = w_avg2 + (all_w3 - w_avg3) * truncation_psi

  # all_w[0][0] = all_w2[0][0]
  # all_w[0][1] = all_w2[0][1]
  # all_w[0][2] = all_w2[0][2]
  # all_w[0][3] = all_w2[0][3]
  # all_w[0][4] = all_w2[0][4]
  # all_w[0][5] = all_w2[0][5]
  # all_w[0][6] = all_w2[0][6]
  if deeper_copy:
    all_w[0][7] = all_w3[0][7]
  all_w[0][8] = all_w3[0][8]
  all_w[0][9] = all_w3[0][9]
  all_w[0][10] = all_w3[0][10]
  all_w[0][11] = all_w3[0][11]
  all_w[0][12] = all_w3[0][12]
  all_w[0][13] = all_w3[0][13]
  all_w[0][14] = all_w3[0][14]
  all_w[0][15] = all_w3[0][15]
  all_w[0][16] = all_w3[0][16]
  all_w[0][17] = all_w3[0][17]
  ##(np.random.randint(-25283287,44716253,(512))/10000000).astype('float32') 

  pics_all_w_list_nst.append(all_w.cpu().numpy())

  all_images = G.synthesis(all_w, noise_mode=noise_mode)
  # print(all_images)
  all_images = (all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
  im = PIL.Image.fromarray(all_images[0], 'RGB')
  
  imgs.append(im)


# IPython.display.display(im)
createImageGrid([orig_im] + imgs, scale=0.4, rows=rows)

In [ ]:
#@title # 5.1 Choose one

import uuid

num = 0 #@param {type:"integer"}
wwww = torch.from_numpy(pics_all_w_list_nst[num]).to(device)
all_images_ = G.synthesis(wwww, noise_mode=noise_mode)
# print(all_images)
all_images_ = (all_images_.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
im_ = PIL.Image.fromarray(all_images_[0], 'RGB')
display(createImageGrid([im_], scale=0.5, rows=1))


save_npz = False #@param {type:"boolean"}
if save_npz:
    np.savez('/content/' + uuid.uuid4().hex + '_projected_w_nst.npz', w=pics_all_w_list_nst[num])



